In [1]:
import numpy as np
import pandas as pd
from ChannelAttribution import *
import plotly.io as pio
pio.renderers.default = 'iframe'

*** Looking to run more advanced attribution? Install ChannelAttribution Pro for free running install_pro(). Visit https://channelattribution.io for more info.
Version: 2.2.4


In [2]:
#pip install --upgrade setuptools
#pip install Cython
#pip install ChannelAttribution
#pip install plotly

In [3]:
#pip install plotly==5.18.0

In [4]:
#Data = pd.read_csv("https://channelattribution.io/csv/Data.csv",sep=";")
Data = pd.read_csv('SampleDataChannelAttribution.csv', index_col=0)

In [5]:
Data.head()

,path,total_conversions,total_conversion_value,total_null
0,eta > iota > alpha > eta,1,0.244,3
1,iota > iota > iota > iota,2,3.195,6
2,alpha > iota > alpha > alpha > alpha > iota > ...,2,6.754,6
3,beta > eta,1,2.402,3
4,iota > eta > theta > lambda > lambda > theta >...,0,0.000,2


In [6]:
DataGroup = Data.groupby('path').agg('sum').reset_index()

In [7]:
DataGroup.head()

,path,total_conversions,total_conversion_value,total_null
0,alpha > alpha > alpha,4197,12452.001957,14413
1,alpha > alpha > alpha > alpha > alpha > alpha,1503,3918.143055,6001
2,alpha > alpha > alpha > alpha > alpha > alpha ...,25,50.935588,108
3,alpha > alpha > alpha > alpha > alpha > alpha ...,4,8.096000,14
4,alpha > alpha > alpha > alpha > alpha > alpha ...,7,9.632000,29


# Heuristic - First Touch, Last Touch, Linear Attribution 

In [8]:
H=heuristic_models(DataGroup,"path","total_conversions", var_value="total_conversion_value", flg_pro=False)

In [9]:
H

,channel_name,first_touch_conversions,first_touch_value,last_touch_conversions,last_touch_value,linear_touch_conversions,linear_touch_value
0,alpha,6308.0,19121.272356,8447.0,28414.214274,7574.718594,24524.709569
1,theta,1606.0,6652.349347,653.0,2799.091988,1022.801394,4295.743619
2,kappa,74.0,305.743250,230.0,1069.384250,137.964078,599.747786
3,eta,3164.0,11909.476213,4167.0,16754.203800,3539.951157,13783.497051
4,iota,4606.0,19597.261277,3355.0,13487.974271,3857.096221,15988.988995
5,gamma,165.0,718.977992,92.0,506.013992,121.041639,569.417358
6,lambda,902.0,3735.602165,1207.0,5249.949985,1035.257572,4430.316169
7,epsilon,99.0,412.301243,531.0,2202.612289,272.170438,1106.270065
8,beta,2831.0,12235.591744,989.0,3850.020987,2083.500145,8954.266717
9,zeta,27.0,103.004000,107.0,453.260750,136.551540,539.528763


# Markov Chain Model

In [10]:
M=markov_model(DataGroup, "path", "total_conversions", var_value="total_conversion_value", flg_pro=False)
M

Number of simulations: 100000 - Convergence reached: 2.52% < 5.00%
Percentage of simulated paths that successfully end before maximum number of steps (46) is reached: 99.99%


,channel_name,total_conversions,total_conversion_value
0,alpha,5269.244558,18525.374466
1,theta,1899.967990,7209.285523
2,kappa,277.723716,1127.609141
3,eta,3378.659126,13043.349149
4,iota,3898.452838,15092.379699
5,gamma,167.009532,644.527409
6,lambda,1281.657419,5205.710158
7,epsilon,619.248826,2435.026621
8,beta,2597.092047,9958.432737
9,zeta,394.067435,1554.221792


In [11]:
M2 = markov_model(Data=DataGroup, var_path= "path", var_conv="total_conversions", 
             var_null = "total_null", var_value="total_conversion_value")
M2

Number of simulations: 100000 - Convergence reached: 3.23% < 5.00%
Percentage of simulated paths that successfully end before maximum number of steps (46) is reached: 99.99%
*** Install ChannelAttribution Pro for free running install_pro(). Visit https://channelattribution.io for more info. Set flg_pro=False to hide this message.


,channel_name,total_conversions,total_conversion_value
0,alpha,5438.636878,18896.580726
1,theta,2045.643665,7656.556677
2,kappa,273.050905,1068.228078
3,eta,3316.897059,13234.299086
4,iota,3854.046380,14865.248785
5,gamma,129.811086,455.101179
6,lambda,1222.014706,4889.539397
7,epsilon,541.625566,2066.226580
8,beta,2538.030543,10057.092926
9,zeta,425.243213,1614.098152


In [12]:
Data.total_conversion_value.sum()

np.float64(74802.97158721084)

In [13]:
M.total_conversion_value.sum()

np.float64(74802.97158721078)

In [14]:
def plot_attrib_conv(H, M):
    
    R=pd.merge(H,M,on="channel_name",how="inner")
    R1=R[["channel_name","first_touch_conversions","last_touch_conversions",\
    "linear_touch_conversions","total_conversions"]]
    R1.columns=["channel_name","first_touch","last_touch","linear_touch","markov_model"]
    R1=pd.melt(R1, id_vars="channel_name")

    data = [dict(
          type = "histogram",
          histfunc="sum",
          x = R1.channel_name,
          y = R1.value,
          transforms = [dict(
            type = "groupby",
            groups = R1.variable,
          )],
        )]

    fig = dict({"data":data})
    pio.show(fig,validate=False)

    return()

In [15]:
def plot_attrib_value(H, M):
    R=pd.merge(H,M,on="channel_name",how="inner")
    R2=R[["channel_name","first_touch_value","last_touch_value",\
    "linear_touch_value","total_conversion_value"]]
    R2.columns=["channel_name","first_touch","last_touch","linear_touch","markov_model"]
    
    R2=pd.melt(R2, id_vars="channel_name")
    
    data = [dict(
      type = "histogram",
      histfunc="sum",
      x = R2.channel_name,
      y = R2.value,
      transforms = [dict(
        type = "groupby",
        groups = R2.variable,
      )],
    )]
    
    fig = dict({"data":data})
    pio.show(fig,validate=False)

    return ()

In [16]:
plot_attrib_conv(H, M)

()

In [17]:
plot_attrib_value(H, M)

()

# On Clickstream Data

In [18]:
# On our data
DF = pd.read_csv('MergedChains.csv', index_col= 0)
DF.head()

,Channel,ConversionCount,Value,NonConversionCount
0,"['fb', 'fb', 'fb', 'fb', 'fb', 'fb']",0.0,0.0,1.0
1,"['fb', 'fb', 'fb', 'fb', 'fb', 'insta']",1.0,171.0,0.0
2,"['fb', 'fb', 'fb', 'fb', 'goog', 'fb']",1.0,58.3,0.0
3,"['fb', 'fb', 'fb', 'fb', 'insta', 'insta', 'fb']",1.0,59.7,0.0
4,"['fb', 'fb', 'fb', 'fb', 'insta']",0.0,0.0,1.0


In [19]:
def cast_list(ll):
    '''
    function to convert clickstream string to required format with > as separator'
    '''
    nll = eval(ll)
    strr = ''
    for x in nll:
        if strr == '':
            strr += x
        else:
            strr += ' > ' + x
    return (strr)

# Test function 
ll = str(['fb', 'fb', 'fb', 'fb', 'fb'])
cast_list(ll)

'fb > fb > fb > fb > fb'

In [20]:
DF['newchannel'] = DF['Channel'].apply(lambda x: cast_list(x))
DF1 = DF[['newchannel', 'ConversionCount', 'Value', 'NonConversionCount']]
DF1.columns = ['path', 'total_conversions', 'total_conversion_value', 'total_null']
DF1.to_csv('TestChains.csv')
DF1.head()

,path,total_conversions,total_conversion_value,total_null
0,fb > fb > fb > fb > fb > fb,0.0,0.0,1.0
1,fb > fb > fb > fb > fb > insta,1.0,171.0,0.0
2,fb > fb > fb > fb > goog > fb,1.0,58.3,0.0
3,fb > fb > fb > fb > insta > insta > fb,1.0,59.7,0.0
4,fb > fb > fb > fb > insta,0.0,0.0,1.0


In [21]:
M=markov_model(DF1, "path", var_conv = "total_conversions", 
               var_null = "total_null", var_value="total_conversion_value", flg_pro=False)
M

Number of simulations: 100000 - Convergence reached: 3.04% < 5.00%
Percentage of simulated paths that successfully end before maximum number of steps (9) is reached: 99.99%


,channel_name,total_conversions,total_conversion_value
0,fb,8351.762626,7.799273e+05
1,insta,12484.858270,1.164228e+06
2,goog,3238.875158,3.001025e+05
3,twit,3033.503946,2.833761e+05


In [22]:
H=heuristic_models(DF1,"path", "total_conversions",var_value="total_conversion_value", flg_pro=False)
H

,channel_name,first_touch_conversions,first_touch_value,last_touch_conversions,last_touch_value,linear_touch_conversions,linear_touch_value
0,fb,6025.0,562159.1,6134.0,573535.1,6082.711111,568123.568254
1,insta,10255.0,951178.0,10123.0,936365.9,10185.762302,943750.620675
2,goog,2091.0,193068.2,2113.0,195342.6,2109.700000,194664.373254
3,twit,8738.0,821228.7,8739.0,822390.4,2017.826587,189278.237817


In [23]:
DF1.total_conversion_value.sum()

np.float64(2527634.0000000033)

In [24]:
M.total_conversion_value.sum()

np.float64(2527634.0000000033)

In [25]:
DF1.total_conversions.sum()

np.float64(27109.0)

In [26]:
M.total_conversions.sum()

np.float64(27108.999999999996)

In [27]:
plot_attrib_conv(H, M)

()

In [28]:
plot_attrib_value(H, M)

()